In [5]:
import MetaTrader5 as mt
import pandas as pd
import datetime
import matplotlib as mp
import yfinance as yf

In [6]:
mt.initialize()

True

In [7]:
account_info = mt.account_info()

In [8]:
posicoes = mt.positions_get()

In [9]:
df_positions = pd.DataFrame(list(posicoes))
print(df_positions)

Empty DataFrame
Columns: []
Index: []


In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime

# Lista de tickers da B3 no formato do Yahoo Finance
df_positions = ['PETR4.SA', 'RAIZ4.SA','GOAU4.SA']

# Datas de início e fim (últimos 10 anos)
end_date = datetime.today()
start_date = end_date.replace(year=end_date.year - 10)

# Dicionário para armazenar os DataFrames por ticker
historical_data = {}

# Loop pelos tickers
for ticker in df_positions:
    print(f"Baixando dados para {ticker}...")
    data = yf.download(ticker, start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d"))
    
    if not data.empty:
        data["Ticker"] = ticker  # adiciona uma coluna com o nome do ticker
        historical_data[ticker] = data
    else:
        print(f"Dados não encontrados para {ticker}.")

# Se quiser juntar tudo em um único DataFrame:
df_all = pd.concat(historical_data.values(), axis=0)



Baixando dados para PETR4.SA...


C:\Users\jacks\AppData\Local\Temp\ipykernel_2224\3089143.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d"))
[*********************100%***********************]  1 of 1 completed
C:\Users\jacks\AppData\Local\Temp\ipykernel_2224\3089143.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d"))


Baixando dados para RAIZ4.SA...


[*********************100%***********************]  1 of 1 completed
C:\Users\jacks\AppData\Local\Temp\ipykernel_2224\3089143.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d"))


Baixando dados para GOAU4.SA...


[*********************100%***********************]  1 of 1 completed


In [2]:
df = df_all.reset_index()

# 2. Renomeia colunas com multi-index (flattening)
df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df.columns]

# 3. Transforma em formato longo
df_long = df.melt(id_vars='Date_', 
                  var_name='variable', 
                  value_name='value')

# 4. Extrai o tipo de preço e o ticker da coluna "variable"
df_long[['price_type', 'ticker']] = df_long['variable'].str.extract(r'(\w+)_(.+)')

# 5. Pivot para que cada tipo de preço vire uma coluna
df_final = df_long.pivot_table(index=['Date_', 'ticker'],
                               columns='price_type',
                               values='value').reset_index()

# 6. Renomeia a coluna Date_
df_final = df_final.rename(columns={"Date_": "date"})

# 7. Reorganiza colunas
df_final = df_final[['date', 'ticker', 'Open', 'High', 'Low', 'Close', 'Volume']]




In [23]:
df_final.to_csv('dadosesperado.csv')

In [ ]:
saldo = account_info.balance

0.96

In [30]:
# Garante que a data está em datetime
df_final['date'] = pd.to_datetime(df_final['date'])

# Cria uma coluna de mês
df_final['mes'] = df_final['date'].dt.to_period('M')

# Pega o último preço de fechamento de cada mês por ticker
df_mensal = df_final.groupby(['ticker', 'mes'])['Close'].last().reset_index()

# Ordena
df_mensal = df_mensal.sort_values(['ticker', 'mes'])

# Calcula o retorno mensal por ticker
df_mensal['retorno_mensal'] = df_mensal.groupby('ticker')['Close'].pct_change()

# Calcula o retorno esperado mensal por ticker
retornos_mensais = df_mensal.groupby('ticker')['retorno_mensal'].mean().reset_index()

# Renomeia e converte para porcentagem
retornos_mensais.columns = ['ticker', 'retorno_esperado_mensal']
retornos_mensais['retorno_esperado_mensal'] *= 100

# Exibe a tabela
